In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install impyute

In [ ]:
import re
import missingno as msno
from impyute.imputation.cs import mice
from scipy import stats
from scipy.stats import norm, skew
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib
import matplotlib.pyplot as plt # for plotting
%matplotlib inline
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
import warnings
warnings.filterwarnings('ignore') # Suppress warnings 

In [ ]:
pd.set_option("max_row", 999)
pd.set_option("max_columns", 999)

In [ ]:
path = "../input/home-credit-default-risk"

In [ ]:
df_train = pd.read_csv(path + "/application_train.csv")
df_previous = pd.read_csv(path + "/previous_application.csv")

In [ ]:
df_bureau = pd.read_csv(path + "/bureau.csv")

In [ ]:
df_previous.dtypes

In [ ]:
df_bureau.head(30)

In [ ]:
df_bureau.describe(include = "all")

In [ ]:
df_bureau.nunique()

In [ ]:
msno.matrix(df=df_previous.iloc[:, :], color=(0.1, 0.6, 0.8))

In [ ]:
msno.matrix(df=df_bureau.iloc[:, :], color=(0.1, 0.6, 0.8))

In [ ]:
df_previous_ratio = df_previous.assign(AMT_MONTH = lambda x : (x["AMT_CREDIT"]/x["CNT_PAYMENT"]))

In [ ]:
df_previous_ratio.info

## CNT_PAYMENT가 대출 기간일수도 있음

In [ ]:
df_previous_corr = df_previous_ratio.loc[:,["AMT_ANNUITY", "AMT_CREDIT", "CNT_PAYMENT", "AMT_MONTH"]]
corrmat_previous_credit = df_previous_corr.corr()
plt.figure(figsize=(20,10))
plt.xticks(rotation = 45)
g = sns.heatmap(corrmat_previous_credit, annot=True, cmap="YlGnBu")

In [ ]:
df_previous.head()

In [ ]:
df_previous.sort_values(by = "SK_ID_CURR")

In [ ]:
df_previous.describe(include = "all")

In [ ]:
df_train.head()

In [ ]:
EDA_df_train = df_train.drop("SK_ID_CURR", axis = 1)

## 기본 인적 사항과 관련된 상관관계

In [ ]:
df_train_basic = df_train.loc[:,["TARGET","NAME_CONTRACT_TYPE","CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY","CNT_CHILDREN","AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY","AMT_GOODS_PRICE","NAME_TYPE_SUITE","NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","REGION_POPULATION_RELATIVE","DAYS_BIRTH","DAYS_EMPLOYED","DAYS_REGISTRATION","DAYS_ID_PUBLISH","OWN_CAR_AGE","FLAG_MOBIL","FLAG_EMP_PHONE","FLAG_WORK_PHONE","FLAG_CONT_MOBILE","FLAG_PHONE","FLAG_EMAIL","OCCUPATION_TYPE","CNT_FAM_MEMBERS","REGION_RATING_CLIENT","REGION_RATING_CLIENT_W_CITY","WEEKDAY_APPR_PROCESS_START","HOUR_APPR_PROCESS_START","REG_REGION_NOT_LIVE_REGION","REG_REGION_NOT_WORK_REGION","LIVE_REGION_NOT_WORK_REGION","REG_CITY_NOT_LIVE_CITY","REG_CITY_NOT_WORK_CITY","LIVE_CITY_NOT_WORK_CITY","ORGANIZATION_TYPE","EXT_SOURCE_1","EXT_SOURCE_2","EXT_SOURCE_3","OBS_30_CNT_SOCIAL_CIRCLE","DEF_30_CNT_SOCIAL_CIRCLE","OBS_60_CNT_SOCIAL_CIRCLE","DEF_60_CNT_SOCIAL_CIRCLE","DAYS_LAST_PHONE_CHANGE","AMT_REQ_CREDIT_BUREAU_HOUR","AMT_REQ_CREDIT_BUREAU_DAY","AMT_REQ_CREDIT_BUREAU_WEEK","AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_YEAR"]]

In [ ]:
corrmat_Train_Basic = df_train_basic.corr()
plt.figure(figsize=(30,20))
plt.xticks(rotation = 45)
g = sns.heatmap(corrmat_Train_Basic, annot=True, cmap="YlGnBu")

## 문서와 관련된 상관관계

In [ ]:
df_train_documents = df_train.loc[:,["FLAG_DOCUMENT_2", 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
                                  "FLAG_DOCUMENT_6", 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
                                  "FLAG_DOCUMENT_10", 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
                                  "FLAG_DOCUMENT_14", 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
                                  "FLAG_DOCUMENT_18", 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'TARGET']]
df_train_documents

In [ ]:
# most correlated features

corrmat_Train_Doc = df_train_documents.corr()
plt.figure(figsize=(30,20))
plt.xticks(rotation = 45)
g = sns.heatmap(corrmat_Train_Doc,annot=True,cmap="YlGnBu")

In [ ]:
msno.matrix(df=df_train_documents.iloc[:, :], color=(0.1, 0.6, 0.8))

## 부동산과 관련된 features의 상관관계

In [ ]:
df_train_house = df_train.loc[:,["TARGET","APARTMENTS_AVG", "BASEMENTAREA_AVG", "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BUILD_AVG", "COMMONAREA_AVG", "ELEVATORS_AVG", "ENTRANCES_AVG", "FLOORSMAX_AVG", "FLOORSMIN_AVG", "LANDAREA_AVG", "LIVINGAPARTMENTS_AVG", "LIVINGAREA_AVG", "NONLIVINGAPARTMENTS_AVG", "NONLIVINGAREA_AVG", "APARTMENTS_MODE", "BASEMENTAREA_MODE" ,"YEARS_BEGINEXPLUATATION_MODE", "YEARS_BUILD_MODE", "COMMONAREA_MODE", "ELEVATORS_MODE", "ENTRANCES_MODE", "FLOORSMAX_MODE", "FLOORSMIN_MODE", "LANDAREA_MODE", "LIVINGAPARTMENTS_MODE", "LIVINGAREA_MODE", "NONLIVINGAPARTMENTS_MODE", "NONLIVINGAREA_MODE", "APARTMENTS_MEDI", "BASEMENTAREA_MEDI", "YEARS_BEGINEXPLUATATION_MEDI", "YEARS_BUILD_MEDI", "COMMONAREA_MEDI", "ELEVATORS_MEDI", "ENTRANCES_MEDI", "FLOORSMAX_MEDI", "FLOORSMIN_MEDI", "LANDAREA_MEDI", "LIVINGAPARTMENTS_MEDI", "LIVINGAREA_MEDI", "NONLIVINGAPARTMENTS_MEDI", "NONLIVINGAREA_MEDI", "FONDKAPREMONT_MODE", "HOUSETYPE_MODE", "TOTALAREA_MODE", "WALLSMATERIAL_MODE", "EMERGENCYSTATE_MODE"]]
df_train_house

In [ ]:
corrmat_Train_House = df_train_house.corr()
plt.figure(figsize=(30,20))
plt.xticks(rotation = 45)
g = sns.heatmap(corrmat_Train_House,annot=True,cmap="YlGnBu")

In [ ]:
dummy = pd.get_dummies(df_train)

In [ ]:
df_train_impute = pd.DataFrame(IterativeImputer(verbose = False).fit_transform(dummy))
df_train_impute.head()

In [ ]:
df_train_impute.to_csv("df_train_전처리완료.csv", index = False, encoding = "utf-8-sig")

In [ ]:
msno.matrix(df=df_train_impute.iloc[:, :], color=(0.1, 0.6, 0.8))

In [ ]:
#변환
from impyute.imputation.cs import mice
np_imputed=mice(df_bureau.values) # mice 학습시작
df_imputed = pd.DataFrame(np_imputed)

# 변환 후 데이터 확인
df_imputed.head(10)

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
df_target = df_train.loc[:,["SK_ID_CURR", "TARGET"]]
df_target.head()

In [ ]:
TB_merge = pd.merge(df_target, df_bureau, how = "inner", on = "SK_ID_CURR")
TB_merge

In [ ]:
TP_merge = pd.merge(df_target, df_previous, how = "inner", on = "SK_ID_CURR")
TP_merge

In [ ]:
def type_features(data):
    categorical_features = data.select_dtypes(include = ["object"]).columns
    numerical_features = data.select_dtypes(exclude = ["object"]).columns
    print( "categorical_features :",categorical_features)
    print('-----'*40)
    print("numerical_features:",numerical_features)

In [ ]:
type_features(df_train)

In [ ]:
C_features = df_train.select_dtypes(include = ["object"])
C_features

In [ ]:
# most correlated features

corrmat_T = df_train.corr()
top_corr_features_T = corrmat_T.index[abs(corrmat_T["TARGET"])>=0.03]
plt.figure(figsize=(30,20))
plt.xticks(rotation = 45)
g = sns.heatmap(df_train[top_corr_features_T].corr(),annot=True,cmap="YlGnBu")
plt.savefig("corr.png")

In [ ]:
TB_merge.head()

In [ ]:
corrmat_TB = TB_merge.corr()
plt.figure(figsize=(30,20))
plt.xticks(rotation = 45)
g = sns.heatmap(corrmat_TB,annot=True,cmap="YlGnBu")

In [ ]:
corrmat_TP = TP_merge.corr()
plt.figure(figsize=(30,20))
plt.xticks(rotation = 45)
g = sns.heatmap(corrmat_TP,annot=True,cmap="YlGnBu")

In [ ]:
from scipy.stats import chi2_contingency
list_C = C_features.columns
list_C

In [ ]:
N_Contract_type = pd.crosstab(df_train['TARGET'], df_train[list_C[0]])
result = chi2_contingency(observed = N_Contract_type)
print("카이 제곱 통계랑 : ", result[0])
print("p-value : ", result[1])

In [ ]:
N_Contract_type

In [ ]:
def make_chi2(frame):
    for list_C in range(C_features.shape[1]):
        for i in range(list_C):
            N_Contract_type = pd.crosstab(frame['TARGET'], frame[[i]])
            result = chi2_contingency(observed = N_Contract_type)
            print("#", i)
            print("카이 제곱 통계랑 : ", result[0])
            print("p-value : ", result[1])

In [ ]:
type_features(TB_merge)

In [ ]:
TB_C_features = TB_merge.select_dtypes(include = ["object"])
TB_C_features

In [ ]:
list_C = TB_C_features.columns
list_C

In [ ]:
make_chi2(TB_merge['TARGET'])

In [ ]:
N_Contract_type = pd.crosstab(TB_merge['TARGET'], TB_merge[list_C[0]])
result = chi2_contingency(observed = N_Contract_type)
print("카이 제곱 통계랑 : ", result[0])
print("p-value : ", result[1])

In [ ]:
def make_chi2(frame):
    for i in range(len(list_C)):
        N_Contract_type = pd.crosstab(frame["TARGET"], frame[list_C[i]])
        result = chi2_contingency(observed = N_Contract_type)
        print("#", i, list_C[i])
        print("카이 제곱 통계랑 : ", result[0])
        print("p-value : ", result[1])

In [ ]:
C_features = df_train.select_dtypes(include = ["object"])
list_C = C_features.columns
list_C

In [ ]:
make_chi2(df_train)

In [ ]:
TP_merge.head()